In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import streamlit as st

1.	wstępna analiza danych i czyszczenie:
a.	duplikaty
b.	wartości odstające
c.	spójność
d.	braki w danych
e.	skala wartości
f.	inne
2.	wizualizacja rozkładu zmiennych, zależności ceny od innych zmiennych, liczebność kategorii
3.	budowa modelu regresji ceny od pozostałych zmiennych. Istotne zmienne należy wybrać eliminacją wsteczną lub selekcją postępującą.
4.	wizualizacja modelu regresji
5.	stworzenie dashboardu z powyższymi wizualizacjami i próbką danych w postaci tabeli. Dashboard powinien umożliwiać zmianę parametrów wykresów - np. zależność ceny od innej, wybranej przez użytkownika zmiennej

Wszystkie wizualizacje i dane powinny zostać zaprezentowane w postaci dashboardu. Etapy czyszczenia danych i budowy modelu regresji należy zawrzeć w notatniku Jupytera (.ipynb).


In [2]:
#porządkowanie tabeli
df = pd.read_csv("messy_data.csv", names=['carat', 'clarity' , 'color' ,'cut', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price'])
df.drop(index=0, inplace=True)

In [3]:
#odrzucenie duplikatów
df.drop_duplicates(subset=None, inplace=True)

In [4]:
#zamiana dziwnych i pustych wartości na NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

In [5]:
#konwersja na typ float
df['carat'] = df['carat'].astype(float)
df['x dimension'] = df['x dimension'].astype(float)
df['y dimension'] = df['y dimension'].astype(float)
df['z dimension'] = df['z dimension'].astype(float)
df['depth'] = df['depth'].astype(float)
df['table'] = df['table'].astype(float)
df['price'] = df['price'].astype(float)

#zamiana na duże litery
df['cut'] = df['cut'].str.upper()
df['clarity'] = df['clarity'].str.upper()
df['color'] = df['color'].str.upper()

In [6]:
#zamiana NaN na średnią tam gdzie się da
df['carat'].fillna(df["carat"].mean(), inplace=True)
df['x dimension'].fillna(df["x dimension"].mean(), inplace=True)
df['y dimension'].fillna(df["y dimension"].mean(), inplace=True)
df['z dimension'].fillna(df["z dimension"].mean(), inplace=True)
df['depth'].fillna(df["depth"].mean(), inplace=True)
df['table'].fillna(df["table"].mean(), inplace=True)
df['price'].fillna(df["price"].mean(), inplace=True)

#i usunięcie tam gdzie się nie da
df.dropna(inplace=True)

In [7]:
# Usuwanie wartości odstających
numeric_columns = ['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table', 'price']

z = np.abs(stats.zscore(df['carat']))
threshold_z = 2
outlier_indices = np.where(z > threshold_z)[0]
no_outliers = df.drop(outlier_indices)
df = no_outliers

In [8]:
import pandas as pd
import streamlit as st
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Convert categorical variables into dummy/indicator variables
df = pd.get_dummies(df, columns=['clarity', 'color', 'cut'], drop_first=True)  # add drop_first=True to avoid dummy variable trap

# Split the data into features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform linear regression
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
st.write("Mean Squared Error:", mse)

# Plot results using Plotly Express
fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Actual Price', 'y': 'Predicted Price'}, title='Actual vs Predicted Price')
st.plotly_chart(fig)


2024-02-03 20:28:12.640 
  command:

    streamlit run C:\Users\przem\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()